In [1]:
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

import re
from string import punctuation
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
path_corpus_tut = Path('../data/corpora/tutby_700306_702020.csv')
path_corpus_tut_clean = Path('../data/corpora/clean_tutby_700306_702020.csv')

In [3]:
%%time

data = pd.read_csv(path_corpus_tut)

corpus = data['document']

print(data.shape)
display(data.head(3))

(1431, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/700306.html,Общество,"Хороводы на перекрестке, водомет и задержания....",2020-09-13T21:33:00+03:00,\n13 сентября в Бресте несколько тысяч человек...,"['акции протеста', 'силовые ведомства']"
1,https://news.tut.by/700307.html,Новость дня,«Покраснело ухо и было разбито колено». В Жоди...,2020-09-13T20:39:00+03:00,"Читатели прислали в редакцию несколько видео, ...","['акции протеста', 'милиция', 'силовые ведомст..."
2,https://news.tut.by/700308.html,Общество,"Многометровый бчб-флаг, «космонавты» у исполко...",2020-09-13T23:21:00+03:00,\nСегодня воскресенье — и гомельчане снова выш...,"['Видео TUT.BY', 'акции протеста', 'местная вл..."


Wall time: 169 ms


In [4]:
russian_stop_words = set(stopwords.words('russian'))
mystem = Mystem() 

def preprocess_text(text):
    text = text.lower()
    text = re.sub(u'\xa0|\n', ' ', text)
    text = re.sub('[^а-яa-z ]', '', text)
    tokens = mystem.lemmatize(text)
    tokens = [token for token in tokens if ((token not in russian_stop_words) and (token.strip() not in punctuation) and (len(token) > 2))]
    text = ' '.join(tokens)
    return text

In [5]:
%%time

separator = 'qsefthukoaxdvgnjthukoaxdvefcngwojd'
batch_size = 1000

result = []

for i in tqdm(range(len(corpus) // batch_size + 1)):
    batch = corpus[i * batch_size:(i + 1) * batch_size]
    batch_joint = f' {separator} '.join(batch)
    batch_clean = preprocess_text(batch_joint)
    batch_clean = batch_clean.split(separator)
    result += batch_clean
    
result = [_.strip() for _ in result]


print(len(corpus), len(result))
print(corpus.iloc[-77], result[-77])

data['document'] = result
data.to_csv(path_corpus_tut_clean, index=False)


1431 1431
25 сентября в микрорайоне Березовка прошел семейный праздник для соседей, а во дворе многоэтажек на Гоголя выступил детский театр.

Праздник в Березовке. Фото: Сергей Никрашевич, Брестская газета

Настоящий семейный праздник устроили для брестчан жители Березовки. Атмосфера во дворе царила праздничная: игры, конкурсы, танцы, сладкий стол. Отдыхали и дети, и взрослые. Приходил даже огромный медведь, пишет «Брестская газета».

Праздник в Березовке. Фото: Сергей Никрашевич, Брестская газета


Праздник в Березовке. Фото: Сергей Никрашевич, Брестская газета


Праздник в Березовке. Фото: Сергей Никрашевич, Брестская газета

В это время между домами № 83 и 85 на улице Гоголя прошла встреча соседей, на которой выступил детский театр, сообщает журнал «Бинокль».

Фото: Бинокль




 сентябрь микрорайон березовка проходить семейный праздник сосед двор многоэтажка гоголь выступать детский театр праздник березовка фото сергей никрашевич брестский газета настоящий семейный праздник устраив

In [6]:
data = pd.read_csv(path_corpus_tut_clean)
print(data.shape)
display(data.head())

(1431, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/700306.html,Общество,"Хороводы на перекрестке, водомет и задержания....",2020-09-13T21:33:00+03:00,сентябрь брест несколько тысяча человек приним...,"['акции протеста', 'силовые ведомства']"
1,https://news.tut.by/700307.html,Новость дня,«Покраснело ухо и было разбито колено». В Жоди...,2020-09-13T20:39:00+03:00,читатель присылать редакция несколько видео ко...,"['акции протеста', 'милиция', 'силовые ведомст..."
2,https://news.tut.by/700308.html,Общество,"Многометровый бчб-флаг, «космонавты» у исполко...",2020-09-13T23:21:00+03:00,сегодня воскресение гомельчанин снова выходить...,"['Видео TUT.BY', 'акции протеста', 'местная вл..."
3,https://news.tut.by/700309.html,Общество,"Взрывы, выстрелы, задержания: что происходило ...",2020-09-13T20:37:00+03:00,минск микрорайон дрозд около проспект победите...,"['Видео TUT.BY', 'акции протеста', 'взрывы']"
4,https://news.tut.by/700310.html,Деньги и власть,"«После инаугурации, если она будет, Лукашенко ...",2020-09-13T23:45:00+03:00,светлана тихановский воскресение вечер проводи...,"['Выборы-2020', 'Президент', 'криминал', 'мили..."
